In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional, TimeDistributed, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error

/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import sklearn.preprocessing as prep

In [3]:
epoch = 10
batch = 256

In [5]:
x_train = np.load('../data/trainX_5_one_stock.npy').astype(float)
y_train= np.load('../data/trainY_5_one_stock.npy').astype(float)
x_test = np.load('../data/testX_5_one_stock.npy').astype(float)
y_test = np.load('../data/testY_5_one_stock.npy').astype(float)

In [6]:
x_train1 = x_train[:, :, 0].reshape(-1, 5, 1)
x_train2 = x_train[:, :, 2].reshape(-1, 5, 1)
x_train1 = np.concatenate((x_train1, x_train2), axis = 2)

mean_x = np.mean(x_train1, axis = 0)
std_x = np.std(x_train1, axis = 0)
x_1 = (x_train1 - mean_x) / std_x
x_test1 = x_test[:, :, 0].reshape(-1, 5, 1)
x_test2 = x_train[:, :, 2].reshape(-1, 5, 1)
x_t1 = (x_test1 - mean_x) / std_x

mean_y = np.mean(y_train)
std_y = np.std(y_train)
y = (y_train - mean_y) / std_y
y_t1 = (y_test - mean_y) / std_y

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(5,2), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.add(Flatten())
model.add(Dense(5,activation='linear'))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")


In [8]:
# model = Sequential()
# model.add(LSTM(128, return_sequences=True,input_shape=(5,6)))
# model.add(Dropout(0.4))
# model.add(LSTM(128))
# model.add(Dropout(0.4))
# model.add(Dense(1, activation='linear'))

# model.summary()
# model.compile(optimizer='adam',loss='mse')#, metrics=['mse'])


In [9]:
filepath="weights.{epoch:02d}-{val_acc:.5f}.h5"
output_path = 'predict_rnn2_{val_acc:.5f}.csv'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', 
                             verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor = 'val_acc', patience=1, verbose=1)
callbacks_list = [checkpoint, early_stop]
history = model.fit(x_1, y,
                batch_size = batch, 
                epochs = 5)
                #validation_split = 0.1,
                #callbacks = callbacks_list)

Epoch 1/5
3996/3996 [==============================] - 2s 496us/step - loss: 0.1817
Epoch 2/5
3996/3996 [==============================] - 1s 186us/step - loss: 0.0196
Epoch 3/5
3996/3996 [==============================] - 1s 181us/step - loss: 0.0070
Epoch 4/5
3996/3996 [==============================] - 1s 182us/step - loss: 0.0043
Epoch 5/5
3996/3996 [==============================] - 1s 181us/step - loss: 0.0038


In [10]:
model.evaluate(x_t1, y_t1)

444/444 [==============================] - 0s 544us/step


0.00335519694983288

In [11]:
y_pred = model.predict(x_t1)

In [12]:
result = y_pred * std_y + mean_y
error = mean_squared_error(y_test, result)
print(error)

9.119543492701098


In [13]:
import pickle
with open('lstm.pickle', 'wb') as handle: # saving the history of the model
    pickle.dump(history.history, handle)